In [47]:
import xml.etree.ElementTree as ET
import urllib2
retmax = 1000 
ids_page_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=bcl2&retmax=" + str(retmax)
ids_xml_page = urllib2.urlopen(ids_page_url).read()

ids_root = ET.fromstring(ids_xml_page)
# write ids to article-ids.txt
with open('raw_data/article-ids.txt', 'w+') as my_file:
    for ids in ids_root.iter('IdList'):
        for id in ids.iter('Id'):
            my_file.write(id.text.encode('utf-8') + '\n')

In [48]:
from bs4 import BeautifulSoup
import re

abstract_url_base = "https://www.ncbi.nlm.nih.gov/pubmed/"
# write urls to article-urls.txt
with open('raw_data/article-content.txt', 'w+') as my_file:
    for ids in ids_root.iter('IdList'):
        for id in ids.iter('Id'):
            # build the full pubmed url
            url = abstract_url_base + id.text.encode('utf-8')
            html_content = urllib2.urlopen(url)
            if(html_content):
                soup = BeautifulSoup(html_content)
                full_text_link = soup.find('a', attrs={'free_status': 'free'})
                if(full_text_link):
                    new_url = full_text_link.attrs['href']
                    req = urllib2.Request(new_url, headers={'User-Agent' : "Magic Browser"})
                    try:
                        html_content = urllib2.urlopen(new_url)
                        soup = BeautifulSoup(html_content)
                        article = ''
                        for p in soup.find_all('p'):
                            article = article + p.text.encode('utf-8')
                        article = article.replace('\r', ' ').replace('\n', ' ').lower()
                        sentences = re.split(r'[\.;]', article)
                        for sentence in sentences:
                            if 'bcl2' in sentence.split():
                                my_file.write(id.text.encode('utf-8') + '\t' + sentence + '\n')
                    except:
                        pass